<a href="https://colab.research.google.com/github/jolhe006/ChatGPT-Tinder-Bot/blob/main/demo/prodia/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio Demo: fake_gan
### This is a fake GAN that shows how to create a text-to-image interface for image generation. Check out the Stable Diffusion demo for more: https://hf.co/spaces/stabilityai/stable-diffusion/
        

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
ENV_VARIABLE="265e3569-427c-40e2-bd6b-211de2d8ad45"

In [13]:
print(ENV_VARIABLE)

265e3569-427c-40e2-bd6b-211de2d8ad45


In [2]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.5 MB/s eta 0:00:00


In [97]:
import numpy as np
import gradio as gr
import requests
import time
import json
import base64
import os
from PIL import Image
from io import BytesIO

class Prodia:
    def __init__(self, api_key, base=None):
        self.base = base or "https://api.prodia.com/v1"
        self.headers = {
            "X-Prodia-Key": api_key
        }

    def generate(self, params):
        response = self._post(f"{self.base}/sdxl/generate", params)
        return response.json()

    def get_job(self, job_id):
        response = self._get(f"{self.base}/job/{job_id}")
        return response.json()

    def wait(self, job):
        job_result = job

        while job_result['status'] not in ['succeeded', 'failed']:
            time.sleep(0.25)
            job_result = self.get_job(job['job'])

        return job_result

    def list_models(self):
        response = self._get(f"{self.base}/sdxl/models")
        return response.json()

    def list_presets(self):
        response = [
          '',
          '3d-model',
          'analog-film',
          'anime',
          'cinematic',
          'comic-book',
          'digital-art',
          'enhance',
          'fantasy-art',
          'isometric',
          'line-art',
          'low-poly',
          'neon-punk',
          'origami',
          'photographic',
          'pixel-art',
          'texture',
          'craft-clay',
          'craft-clay'
        ]
        return response

    def list_samplers(self):
        response = self._get(f"{self.base}/sdxl/samplers")
        return response.json()

    def _post(self, url, params):
        headers = {
            **self.headers,
            "Content-Type": "application/json"
        }
        response = requests.post(url, headers=headers, data=json.dumps(params))

        if response.status_code != 200:
            raise Exception(f"Bad Prodia Response: {response.status_code}")

        return response

    def _get(self, url):
        response = requests.get(url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Bad Prodia Response: {response.status_code}")

        return response


def image_to_base64(image_path):
    # Open the image with PIL
    with Image.open(image_path) as image:
        # Convert the image to bytes
        buffered = BytesIO()
        image.save(buffered, format="PNG")  # You can change format to PNG if needed

        # Encode the bytes to base64
        img_str = base64.b64encode(buffered.getvalue())

    return img_str.decode('utf-8')  # Convert bytes to string



#prodia_client = Prodia(api_key=os.getenv("PRODIA_API_KEY"))
prodia_client = Prodia(api_key=ENV_VARIABLE)

def flip_text(prompt, negative_prompt, model, style_preset, steps, sampler, cfg_scale, width, height, seed):
    result = prodia_client.generate({
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "model": model,
        "style_preset": style_preset,
        "steps": steps,
        "sampler": sampler,
        "cfg_scale": cfg_scale,
        "width": width,
        "height": height,
        "seed": seed
    })

    job = prodia_client.wait(result)

    return job["imageUrl"]

def add_image(gallery, image_output):
    images = gallery.value
    images.append(image_output.value)
    print(image_output.value)
    return images

css = """
#generate {
    height: 100%;
}
"""

with gr.Blocks(css=css) as demo:


    with gr.Row():
        with gr.Column(scale=6):
            gr.Markdown(elem_id="powered-by-prodia", value="Stable Diffusion Web UI for SDXL V1.0.<br>Powered by [Prodia](https://prodia.com).")
            model = gr.Dropdown(interactive=True,value="realismEngineSDXL_v10.safetensors [af771c3f]", show_label=True, label="Stable Diffusion Checkpoint", choices=prodia_client.list_models())
            preset = gr.Dropdown(interactive=True,value="photographic", show_label=True, label="Style Preset", choices=prodia_client.list_presets())

    with gr.Tab("txt2img"):
        with gr.Row():
            with gr.Column(scale=3):
                with gr.Tab("Generation"):
                    with gr.Row():
                        with gr.Column(scale=1):
                            sampler = gr.Dropdown(value="DPM++ 2M Karras", show_label=True, label="Sampling Method", choices=prodia_client.list_samplers())

                        with gr.Column(scale=1):
                            steps = gr.Slider(label="Sampling Steps", minimum=1, maximum=50, value=30, step=1)

                    with gr.Row():
                        with gr.Column(scale=1):
                            width = gr.Slider(label="Width", minimum=512, maximum=1536, value=768, step=8)
                            height = gr.Slider(label="Height", minimum=512, maximum=1536, value=1024, step=8)
                            gr.Markdown(elem_id="resolution", value="*Resolution Maximum: 1MP (1048576 px)*")

                        with gr.Column(scale=1):
                            batch_size = gr.Slider(label="Batch Size", maximum=1, value=1)
                            batch_count = gr.Slider(label="Batch Count", maximum=1, value=1)

                    cfg_scale = gr.Slider(label="CFG Scale", minimum=1, maximum=20, value=7, step=1)
                    seed = gr.Number(label="Seed", value=-1)

        with gr.Row():
            with gr.Tab("Prompts"):
                with gr.Column(scale=6, min_width=600):
                    prompt = gr.Textbox("beautiful, female, ultrarealistic, epic shot, dynamic angle, 4k, masterpiece, best quality, extremely intricate, realistic, sharp focus, award winning, cinematic lighting, extremely detailed, RAW, Detailed face: 1.2, Detailed facial features, detailed expressions, HDR, 8k, sharp focus, complex elements", placeholder="Prompt", show_label=False, lines=3)
                    negative_prompt = gr.Textbox(placeholder="Negative Prompt", show_label=False, lines=3, value="easynegative, ng_deepnegative_v1_75t, (worst quality:2) , (low quality:2) , (normal quality:2) , lowres, bad anatomy, (((bad hands) )), normal quality, ((monochrome) ), ((grayscale) ), ((watermark) ), bad face, poorly drawn face, bad nose, bad lips, (((deformed iris) )), (((deformed pupils) )), (easynegative) , (((bad eyes) )), (mutated hands:1.5) , (mutated fingers:1.5) , (bad-hands-5) , multiple hand, missing hand, poorly drawn hand, extra limbs, duplicate, ((too many fingers) ), ((fused fingers) ), bad arm, distorted arm, extra arms, fused arms, extra legs, missing leg, disembodied leg, extra nipples, detached arm, liquid hand, inverted hand, disembodied limb, bad feet, ((missing fingers) ), extra digit, fewer digits, (interlocked fingers:1.2)")
                with gr.Column():
                    text_button = gr.Button("Generate", variant='primary', size='lg', elem_id="generate")
        with gr.Row():
            with gr.Column(scale=6, min_width=600):
                image_output = gr.Image(interactive=False)
                gallery = gr.Gallery(
                                label="Generated images", show_label=False, elem_id="gallery"
                            , columns=[3], rows=[1], object_fit="contain", height="auto", interactive=False)

        text_button.click(flip_text, inputs=[prompt, negative_prompt, model, preset, steps, sampler, cfg_scale, width, height, seed], outputs=image_output)
        image_output.change(add_image, inputs=[gallery, image_output], outputs=gallery)

demo.queue(max_size=32, api_open=False).launch(debug=True, max_threads=128)
text_button.click()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fc1c861933af5f2c2a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 258, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1684, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1250, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7867 <> https://fc1c861933af5f2c2a.gradio.live


{}

In [96]:
image_output
